In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Plese intsall easysict as  
!pip install easydict

In [3]:
#!/usr/bin/env python
# coding: utf-8

# In[40]:


#  Created by Soniya  on 31/10/18.
#
import torch
import numpy
import scipy
import scipy.io as sio
import torch.utils.data as data_utils
import torch.nn as nn
# import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import math
import argparse
import os
# import shutil
import easydict
import math
import shutil
import time
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
# import torchvision.transforms as transforms
import torchvision.datasets as datasets



# ###------------------------------------------------------
# torch.manual_seed(2019)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed_all(2019)

# ###------------------------------------------------------
### Parameters initailization

total_blocks = 4

#growth_base = 20;
layers_base = 0

#F_max = 80; F_min = 0;
L_max = 10; L_min = 0

#L_length = L_max  L_min
#F_length = F_max - F_min
#length = L_length + F_length

# length = L_max - layers_base

# length = (L_max - layers_base) + 3*(L_max - layers_base)   ## 3 bit for binary to decimal coding
# print('string_length', length)


num_connect = []; num_c = 0;
for i in range(L_max - layers_base):
    num_c  = num_c  + ((L_max - layers_base) - (i+1))

z1 = (L_max - layers_base)
length = (L_max - layers_base) + 3*(L_max - layers_base) + num_c;     # total length of string for complete one block + C_fixed
      
# fixed_bits = num_block*(D-1)
print('total_nodes: L_max - layers_base', (L_max - layers_base))  ## Total nodes
print('3*(L_max - layers_base)', 3*(L_max - layers_base))         ## denote eoperation of each node by 3 bits
print('num_c', num_c)                                             ## total connections within the block
print('string_length', length)

total_nodes: L_max - layers_base 10
3*(L_max - layers_base) 30
num_c 45
string_length 85


In [4]:
args = easydict.EasyDict({
    "alpha": 0.9,
    
    "layers": z1,
    "growth_rate": 6,
    "num_classes": 2,
    
    "layers_base": layers_base,
    "L_max": L_max,
    "L_min": L_min,

    "epochs": 50,
    "batch_size": 64,
    "pop_size": 5,
    "start_epoch":0,
    "lr": 0.0001,
    "momentum": 0.9,
    "weight_decay": 1e-4,
    "print-freq": 500,
    "droprate":0.2,
    "reduce":0.5,
    
#     "resume": '/content/drive/My Drive/Normal_Op_DNet/F1_T1/best_cGA_checkpoint.pth.tar2',
#     "cGA_resume": True,
    "resume": False,
    "cGA_resume": False,
    "bottleneck": True,
    "fixed_flag": True,
#     "name": '/content/drive/My Drive/Normal_Op_DNet/F1_T1/',

    
    "nodes_op": [],
    "connections_from_node": [], 
    "num_connections" : [],
    "block_connections_binary" : [], 
    "nodes_in_block" : [],
    "connections_per_node" : [], 
    "node_num" : [],
})



args1 = easydict.EasyDict({
    "layers": z1,
    "nodes_op": [],
    
    "connections_from_node": [], 
    "num_connections" : [],
    "block_connections_binary" : [], 
    "nodes_in_block" : [],
    "connections_per_node" : [], 
    "node_num" : [],
})

In [5]:
# -----------------------------------------------------------------------------------------
#mat_contents =  sio.loadmat('/home/baadalvm/soniya/COIL100Data/C100Train_Data.mat')
mat_contents =  sio.loadmat('/content/drive/My Drive/Malaria/PW_Fold4/fold4_trdata.mat')
mat_data= mat_contents['data_tr']
pydata = mat_data.transpose(3,2,0,1)
Train_Data = torch.FloatTensor(pydata)

#mat_labels =  sio.loadmat('/home/baadalvm/soniya/COIL100Data/C100Train_labels.mat')
mat_labels =   sio.loadmat('/content/drive/My Drive/Malaria/PW_Fold4/fold4_trdata.mat')
labels= mat_labels['labels_tr']
pylabels = labels.transpose(1,0)
c = pylabels.astype(int)
py1 = torch.LongTensor(c)
Train_labels = py1.view(py1.size(0))

#mat_contents =  sio.loadmat('/home/baadalvm/soniya/COIL100Data/C100Train_Data.mat')
mat_contents =  sio.loadmat('/content/drive/My Drive/Malaria/PW_Fold4/fold4_tedata.mat')
mat_data= mat_contents['data_te']
pydata = mat_data.transpose(3,2,0,1)
Test_Data = torch.FloatTensor(pydata)

#mat_labels =  sio.loadmat('/home/baadalvm/soniya/COIL100Data/C100Train_labels.mat')
mat_labels =   sio.loadmat('/content/drive/My Drive/Malaria/PW_Fold4/fold4_tedata.mat')
labels= mat_labels['labels_te']
pylabels = labels.transpose(1,0)
c = pylabels.astype(int)
py1 = torch.LongTensor(c)
Test_labels = py1.view(py1.size(0))

train_dataset = torch.utils.data.TensorDataset(Train_Data, Train_labels)
test_dataset = torch.utils.data.TensorDataset(Test_Data, Test_labels)

# --- load datasets
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=args.batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=args.batch_size, shuffle=False)

In [6]:
class BottleneckBlock(nn.Module):
    def __init__(self, op, in_planes, out_planes, num_nodes, binary, Cf_out, out_list, init_in_planes, i, dropRate=0.0):
        super(BottleneckBlock, self).__init__() 
#         print('in_planes', in_planes)

        inter_planes = out_planes * 2 #4
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, inter_planes, kernel_size=1, stride=1,
                               padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(inter_planes)
        
        
        if op == 0:
           # print('conv1x1')
            self.node2 = nn.Sequential(
                         nn.Conv2d(inter_planes, out_planes, kernel_size=1, stride=1, padding=0, dilation=1, bias = False)
                        )
        elif op  == 1:
           # print('conv3x3')
            self.node2 = nn.Sequential(
                         nn.Conv2d(inter_planes, out_planes, kernel_size=3, stride=1, padding=1, dilation=1, bias = False)
                        )
        elif op  == 2:  
           # print('d_conv3x3')
            self.node2 = nn.Sequential(
                         nn.Conv2d(inter_planes, out_planes, kernel_size=3, stride=1, padding=2, dilation=2, bias = False)
                        )
        elif op == 3:
           # print('2_conv3x3')
            self.node2 = nn.Sequential(
                         nn.Conv2d(inter_planes, inter_planes, kernel_size=3, stride=1, padding=1, dilation=1, bias = False),
                         nn.Conv2d(inter_planes, out_planes, kernel_size=3, stride=1, padding=1, dilation=1, bias = False)
                        )
        elif op == 4:
           # print('3_conv3x3')
            self.node2 = nn.Sequential(
                         nn.Conv2d(inter_planes, inter_planes, kernel_size=3, stride=1, padding=1, dilation=1, bias = False),
                         nn.Conv2d(inter_planes, inter_planes, kernel_size=3, stride=1, padding=1, dilation=1, bias = False),
                         nn.Conv2d(inter_planes, out_planes, kernel_size=3, stride=1, padding=1, dilation=1, bias = False)
                        )
        elif op  == 5:
            # print('Sep3x3_conv')
            self.node2 = nn.Sequential(
                         nn.Conv2d(inter_planes,inter_planes,kernel_size=3,stride=1,padding=1,dilation=1,groups=inter_planes,bias=False),  
                         nn.Conv2d(inter_planes,out_planes,kernel_size=1,stride=1,padding=0,dilation=1,groups=1,bias=False)
                        )
        elif op  == 6:
           # print('Sep5x5_conv')
            self.node2 =  nn.Sequential(
                          nn.Conv2d(inter_planes,inter_planes,kernel_size=5,stride=1,padding=2,dilation=1,groups=inter_planes,bias=False),
                          nn.Conv2d(inter_planes,out_planes,kernel_size=1,stride=1,padding=0,dilation=1,groups=1,bias=False)
                        )
        elif op  == 7:
           # print('Sep7x7_conv')
            self.node2 =  nn.Sequential(
                          nn.Conv2d(inter_planes,inter_planes,kernel_size=7,stride=1,padding=3,dilation=1,groups=inter_planes,bias=False),
                          nn.Conv2d(inter_planes,out_planes,kernel_size=1,stride=1,padding=0,dilation=1,groups=1,bias=False)
                        )
        
        self.droprate = dropRate
        self.out_planes = out_planes
        self.binary = binary
        self.Cf_out =  Cf_out
        self.out_list =  out_list
        self.init_in_planes = init_in_planes
        self.num_nodes = num_nodes

    
    def forward(self, x):
        # print('NB_x_size_incoming', x.size())
#         print('self.init_in_planes', self.init_in_planes)
#         print('out_planes', self.out_planes)
        m1 = x[:,0:self.init_in_planes,:,:]
#         print('m1', m1.size())
#         print('binary', self.binary)
        
        k1 = 0
        if len(self.binary) > 1:
            for i in range(len(self.binary)):
#                 print('binary[i]', self.binary[i])
                if self.binary[i] == 1:
                    m2 = x.narrow(1, self.init_in_planes+k1, self.out_planes)
#                     print('m2', m2.size())
                    m1  = torch.cat([m1, m2], 1)
#                     print('m1', m1.size())
                    
                k1 = k1 + self.out_planes
#                 print('k1', k1)
#             print('m1', m1.size())
            out = self.conv1(self.relu(self.bn1(m1)))    
        else:
            out = self.conv1(self.relu(self.bn1(x)))
#             print('out',out.size())
        
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
        out = self.node2(self.relu(self.bn2(out)))
        # print('NB_out.size:', out.size())
        
        x = torch.cat([x, out], 1)
#         print('x_outgoing', x.size())
                                
        return x

In [7]:
class TransitionBlock(nn.Module):
    def __init__(self, op,  binary, Cf_out, out_list, num_nodes, in_planes, out_planes,  growth_rate, i, init_in_planes, dropRate=0.0):
        super(TransitionBlock, self).__init__() 
#         print('in_planes', in_planes)

        inter_planes = out_planes * 2 #4
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, inter_planes, kernel_size=1, stride=1,
                               padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(inter_planes)
        
        
        if op == 0:
           # print('conv1x1')
            self.node2 = nn.Sequential(
                         nn.Conv2d(inter_planes, out_planes, kernel_size=1, stride=1, padding=0, dilation=1, bias = False)
                        )
        elif op  == 1:
           # print('conv3x3')
            self.node2 = nn.Sequential(
                         nn.Conv2d(inter_planes, out_planes, kernel_size=3, stride=1, padding=1, dilation=1, bias = False)
                        )
        elif op  == 2:  
           # print('d_conv3x3')
            self.node2 = nn.Sequential(
                         nn.Conv2d(inter_planes, out_planes, kernel_size=3, stride=1, padding=2, dilation=2, bias = False)
                        )
        elif op == 3:
           # print('2_conv3x3')
            self.node2 = nn.Sequential(
                         nn.Conv2d(inter_planes, inter_planes, kernel_size=3, stride=1, padding=1, dilation=1, bias = False),
                         nn.Conv2d(inter_planes, out_planes, kernel_size=3, stride=1, padding=1, dilation=1, bias = False)
                        )
        elif op == 4:
           # print('3_conv3x3')
            self.node2 = nn.Sequential(
                         nn.Conv2d(inter_planes, inter_planes, kernel_size=3, stride=1, padding=1, dilation=1, bias = False),
                         nn.Conv2d(inter_planes, inter_planes, kernel_size=3, stride=1, padding=1, dilation=1, bias = False),
                         nn.Conv2d(inter_planes, out_planes, kernel_size=3, stride=1, padding=1, dilation=1, bias = False)
                        )
        elif op  == 5:
            # print('Sep3x3_conv')
            self.node2 = nn.Sequential(
                         nn.Conv2d(inter_planes,inter_planes,kernel_size=3,stride=1,padding=1,dilation=1,groups=inter_planes,bias=False),  
                         nn.Conv2d(inter_planes,out_planes,kernel_size=1,stride=1,padding=0,dilation=1,groups=1,bias=False)
                        )
        elif op  == 6:
           # print('Sep5x5_conv')
            self.node2 =  nn.Sequential(
                          nn.Conv2d(inter_planes,inter_planes,kernel_size=5,stride=1,padding=2,dilation=1,groups=inter_planes,bias=False),
                          nn.Conv2d(inter_planes,out_planes,kernel_size=1,stride=1,padding=0,dilation=1,groups=1,bias=False)
                        )
        elif op  == 7:
           # print('Sep7x7_conv')
            self.node2 =  nn.Sequential(
                          nn.Conv2d(inter_planes,inter_planes,kernel_size=7,stride=1,padding=3,dilation=1,groups=inter_planes,bias=False),
                          nn.Conv2d(inter_planes,out_planes,kernel_size=1,stride=1,padding=0,dilation=1,groups=1,bias=False)
                        )
        
        self.droprate = dropRate
        self.out_planes = out_planes
        self.binary = binary
        self.Cf_out =  Cf_out
        self.out_list =  out_list
        self.init_in_planes = init_in_planes
        self.num_nodes = num_nodes

    
    def forward(self, x):
        # print('RB_x_size_incoming', x.size())
#         print('self.init_in_planes', self.init_in_planes)
#         print('out_planes', self.out_planes)
        m1 = x[:,0:self.init_in_planes,:,:]
#         print('m1', m1.size())
#         print('binary', self.binary)
        
        k1 = 0
        if len(self.binary) > 1:
            for i in range(len(self.binary)):
#                 print('binary[i]', self.binary[i])
                if self.binary[i] == 1:
                    m2 = x.narrow(1, self.init_in_planes+k1, self.out_planes)
#                     print('m2', m2.size())
                    m1  = torch.cat([m1, m2], 1)
#                     print('m1', m1.size())
                    
                k1 = k1 + self.out_planes
#                 print('k1', k1)
#             print('m1', m1.size())
            out = self.conv1(self.relu(self.bn1(m1)))    
        else:
            out = self.conv1(self.relu(self.bn1(x)))
#             print('out',out.size())
        
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
        out = self.node2(self.relu(self.bn2(out)))
        # print('RB_out.size:', out.size())
        
        x = torch.cat([x, out], 1)
#         print('x_outgoing', x.size())
                                
        return x

In [8]:
# WORKING............
# class TransitionBlock(nn.Module):
#     def __init__(self, in_planes, out_planes, dropRate=0.0):
#         super(TransitionBlock, self).__init__()
#         self.bn1 = nn.BatchNorm2d(in_planes)
#         self.relu = nn.ReLU(inplace=True)
#         self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=2,
#                                padding=0, bias=False)
#         self.droprate = dropRate
#     def forward(self, x):
#         out = self.conv1(self.relu(self.bn1(x)))
#         if self.droprate > 0:
#             out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
#         return F.avg_pool2d(out, 2)
    
# ####----------------------------------------------------------------------
class DenseBlock_NB(nn.Module):
    def __init__(self, nodes_op, bC_binary,  Cf_out, out_list, in_planes, growth_rate, num_nodes, block, stride_extra_conv, dropRate=0.0):
        super(DenseBlock_NB, self).__init__()
#         print('nb_layers', nb_layers.size())
        self.layer = self._make_layer(block, nodes_op, bC_binary, Cf_out, out_list, in_planes, growth_rate, num_nodes, dropRate)
        
        in_planes = int(in_planes +  len(nodes_op)*growth_rate)
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.extra_conv = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride_extra_conv, padding=2, bias=False)
    
    def _make_layer(self, block, nodes_op, bC_binary, Cf_out, out_list, in_planes, growth_rate, num_nodes, dropRate):
        layers = []; init_in_planes = in_planes;
#         print('num_nodes', num_nodes)
        for i in range(num_nodes):
#                 print('i',i)
#                 m1 = nodes_in_block[i]
                if i == 0:
                    layers.append(block(nodes_op[i], in_planes, growth_rate, num_nodes, bC_binary[i+1],  Cf_out, out_list, init_in_planes, i, dropRate))      
                else:
                    in_planes = init_in_planes
#                     print('in_planes_before', in_planes)
#                     print('bC_binary', len(bC_binary))
                    binary = bC_binary[i]
#                     print('binary', binary)
                    for i in range(len(binary)):
#                          print('binary[i]', self.binary[i])
                          if binary[i] == 1:
                              in_planes = in_planes  + growth_rate
#                               print('in_planes', in_planes)
                    
                    layers.append(block(nodes_op[i], in_planes, growth_rate, num_nodes, bC_binary[i+1],  Cf_out, out_list, init_in_planes, i, dropRate))
#                 print('DN')

#         self.extra_conv = nn.Conv2d(in_planes, growth_rate, kernel_size=3, stride=1, padding=2, bias=False)
        return nn.Sequential(*layers)
    
    def forward(self, x):
#         out, out1 = self.layer(x)
#         print('out_size in DenseBlock', out1.size())
#         return out, out1
        out = self.layer(x)
#         print('out', out.size())
        out = self.extra_conv(self.conv1(out))
        
        return out# self.layer(x)



# ####----------------------------------------------------------------------
class DenseBlock_RB(nn.Module):
    def __init__(self, nodes_op, bC_binary,  Cf_out, out_list, num_nodes, in_planes, out_planes, growth_rate, TransitionBlock, reduction, stride_extra_conv, dropRate=0.0):
        super(DenseBlock_RB, self).__init__()
#         print('nb_layers', nb_layers.size())
        self.layer = self._make_layer(TransitionBlock, nodes_op, bC_binary, Cf_out, out_list, in_planes, out_planes, growth_rate, num_nodes, dropRate)
#         in_planes = int(in_planes +  len(nodes_op)*growth_rate)
        
        in_planes = int(in_planes +  len(nodes_op)*int(math.floor(in_planes*reduction)))
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.extra_conv = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride_extra_conv, padding=2, bias=False)
        
        self.droprate = dropRate
    
    def _make_layer(self, TransitionBlock, nodes_op, bC_binary, Cf_out, out_list, in_planes, out_planes, growth_rate, num_nodes, dropRate):
        layers = []; init_in_planes = in_planes;
        # print('in_planes', in_planes)
        for i in range(num_nodes):
#                 print('i',i)
#                 m1 = nodes_in_block[i]
                if i == 0:
                    layers.append(TransitionBlock(nodes_op[i], bC_binary[i+1], Cf_out, out_list, num_nodes, in_planes, out_planes, growth_rate, i, init_in_planes, dropRate))      
                else:
                    in_planes = init_in_planes
#                     print('in_planes_before', in_planes)
#                     print('bC_binary', len(bC_binary))
                    binary = bC_binary[i]
#                     print('binary', binary)
                    for i in range(len(binary)):
#                         print('binary[i]', binary[i])
                        if binary[i] == 1:
                            in_planes = in_planes  + out_planes #growth_rate
#                               print('in_planes', in_planes)
                    
                    layers.append(TransitionBlock(nodes_op[i], bC_binary[i+1], Cf_out, out_list, num_nodes, in_planes, out_planes, growth_rate, i, init_in_planes, dropRate))      
#                 print('DN')
        return nn.Sequential(*layers)
    
    
    def forward(self, x):
#         out, out1 = self.layer(x)
#         print('out_size in DenseBlock', out1.size())
#         return out, out1
        out = self.layer(x)
#         print('out', out.size())
        out = self.extra_conv(self.conv1(out))
    
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
        out = F.avg_pool2d(out, 2)
    
        return out

In [9]:
class DenseNet3(nn.Module):
    def __init__(self, order, nodes_op, bC_binary,  Cf_out, out_list, nodes_num, stride_extra_conv, stride_init_conv,  
                 num_classes, growth_rate, reduction=0.1, bottleneck=True, dropRate=0.0):
        super(DenseNet3, self).__init__()
        
        block_num = 0; num_pool = 0; j = 0; trans_num = 0; #num_relu = 0; num_BN = 0;
        self.CC1 = nn.Sequential()
        in_planes = 2 * growth_rate
        block = BottleneckBlock

        self.CC1.add_module('conv1', nn.Conv2d(3, in_planes, kernel_size=3, stride=stride_init_conv, padding=0, bias=False))
        for i in range(len(order)):
            if order[i] == 1:
                self.CC1.add_module('block'+str(block_num), DenseBlock_NB(nodes_op[0],  bC_binary[0], Cf_out, out_list, in_planes, growth_rate, len(nodes_op[0]), block, stride_extra_conv, dropRate))
                in_planes = int(in_planes +  len(nodes_op[0])*growth_rate)
                block_num = block_num + 1
            else:
                self.CC1.add_module('trans'+str(trans_num), DenseBlock_RB(nodes_op[1],  bC_binary[1], Cf_out, out_list, len(nodes_op[1]), in_planes, int(math.floor(in_planes*reduction)), growth_rate, TransitionBlock, reduction, stride_extra_conv, dropRate=dropRate))
                in_planes = int(in_planes +  len(nodes_op[1])*int(math.floor(in_planes*reduction)))
                trans_num = trans_num + 1

        self.in_planes = in_planes
        self.CC1.add_module('bn1', nn.BatchNorm2d(in_planes))
        self.CC1.add_module('relu', nn.ReLU(inplace=True))
        
#         self.classifier = nn.Sequential()
#         self.classifier.add_module('bn1', nn.BatchNorm2d(in_planes))
#         self.classifier.add_module('relu', nn.ReLU(inplace=True))
#         self.classifier.add_module('pool', nn.AvgPool2d(2, stride=1)) # = F.avg_pool2d(out, 2)
#         self.classifier.add_module('fc', nn.Linear(inplanes_size, num_classes))
#         self.in_planes = inplanes_size
        
        self.fc = nn.Linear(in_planes, num_classes)
        
#         # global average pooling and classifier
#         self.bn1 = nn.BatchNorm2d(in_planes)
#         self.relu = nn.ReLU(inplace=True)
#         self.fc = nn.Linear(in_planes, num_classes)
#         self.in_planes = in_planes
                
    def forward(self, x):
#             out = self.conv1(x)
        out = self.CC1(x)
        # print('out_CC1', out.size())
        out = F.avg_pool2d(out, out.size(2))
        # print('out_pool', out.size())
        out = out.view(-1, self.in_planes)
        # print('out_view', out.size())
        out = self.fc(out)
        # print('out_fc', out.size())
        
        return out

In [10]:
# args.order = [1, 1, 0, 1]

# args.nodes_op  = [[0, 6, 4, 6],  [3, 1, 7, 7, 3, 2, 1]]    # node operation 5 represents depth_sep_conv with filters of size 3x3

# args.block_connections_binary = [[[1], [1], [0, 1], [0, 0, 1]],
#  [[1],
#   [1],
#   [0, 1],
#   [0, 1, 0],
#   [1, 0, 0, 0],
#   [0, 0, 1, 1, 0],
#   [0, 1, 1, 0, 0, 1]]] 

# args.stride_extra_conv = 1
# args.stride_init_conv = 2

# Cf_out = []; out_list = []; #order = args.sequence #[0, 1, 1, 0] #args.sequence #
# model = DenseNet3(args.order, args.nodes_op, args.block_connections_binary, Cf_out, out_list, len(args.nodes_op), 
#   args.stride_extra_conv, args.stride_init_conv, args.num_classes, args.growth_rate)

# print('Number of model parameters: {}'.format(sum([p.data.nelement() for p in model.parameters()])))

In [11]:
args.order = [1, 1, 1, 1]
args.nodes_op  =  [[4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
                   [4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
                   [4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
                   [4, 4, 4, 4, 4, 4, 4, 4, 4, 4],]
       # node operation 5 represents depth_sep_conv with filters of size 3x3

args.block_connections_binary = [[torch.tensor([]),
                  torch.tensor([1.]),
                  torch.tensor([1., 1.]),
                  torch.tensor([1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1., 1., 1., 1., 1.])],
                  [torch.tensor([]),
                  torch.tensor([1.]),
                  torch.tensor([1., 1.]),
                  torch.tensor([1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1., 1., 1., 1., 1.])],
                  [torch.tensor([]),
                  torch.tensor([1.]),
                  torch.tensor([1., 1.]),
                  torch.tensor([1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1., 1., 1., 1., 1.])], [torch.tensor([]),
                  torch.tensor([1.]),
                  torch.tensor([1., 1.]),
                  torch.tensor([1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1., 1., 1., 1.]),
                  torch.tensor([1., 1., 1., 1., 1., 1., 1., 1., 1.])]]

args.stride_extra_conv = 2
args.stride_init_conv = 1

Cf_out = []; out_list = []; #order = args.sequence #[0, 1, 1, 0] #args.sequence #
model = DenseNet3(args.order, args.nodes_op, args.block_connections_binary, Cf_out, out_list, len(args.nodes_op), 
  args.stride_extra_conv, args.stride_init_conv, args.num_classes, args.growth_rate)

print('Number of model parameters: {}'.format(sum([p.data.nelement() for p in model.parameters()])))

Number of model parameters: 1433894


In [12]:
# Train_Data.size()

In [13]:
x = Train_Data[0]
target = Train_labels[0]
x.size()
# target

torch.Size([3, 100, 100])

In [14]:
y = torch.unsqueeze(x, 0)
y.size()

torch.Size([1, 3, 100, 100])

In [15]:
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), args.lr, momentum=args.momentum,
                    nesterov=True, weight_decay=args.weight_decay)

end = time.time()

# model.cuda()

target = target.cuda() #(async=True)
input = y.cuda()
input_var = torch.autograd.Variable(input)
target_var = torch.autograd.Variable(target)
        
        # compute output
output = model(input_var)
# loss = criterion(output, target_var)

start = time.time()

total_time = start - end 
total_time

0.04220175743103027